<a href="https://colab.research.google.com/github/alex-sousa/gerador-grafo/blob/master/gerador_rede_transacoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install mimesis

In [ ]:
import numpy as np

n, p = 10, .05  # number of trials, probability of each trial
s = np.random.binomial(n, p)
s

0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Implementação

## Implementação dos Nós

### Populações e Probabilidades

In [ ]:
import math
import threading
import time
import os

populacao = 10000
# proporcoes em relação a populacao

statistics = {
	"pj": 0.05,
	"ub": 0.005
}

# quantidades

counts = {
	"PF" : populacao,
	"PJ" : math.ceil(statistics['pj'] * populacao),
	"UB" : math.ceil(statistics['ub'] * populacao)	,
}

# Probabilidades de Relacionamentos entre pf-pf,pf-pj,pf-pj,pj-pf
p = "0.004,0.01,0.3,0.005"
#seleção de threads
passos="nos,arestas,transacoes,patterns"

probs = list(map(lambda x : float(x), p.split(',')))
steps = set(map(lambda x: x, passos.split(',')))
batchSize = 10000

p_cnae    = 0.05  # probabilidade do cnae estar entre os visados por ocrim
p_valor   = 0.05  # probabilidade de transação valor entre 10.000 e 1.000.000  (ajustar aos valores do COAF)

In [ ]:
import uuid
# classe base para os nós
class Node:
	def toRow(self, headers):
		row = []
    # agrupa campos separados por pipe
		for field in headers: row.append(str(getattr(self, field)))
		return '|'.join(row)

#### Pessoa Física

Implementar:

[ ] Renda de acordo com faixas do IBGE

[ ] Pessoa Politicamente Exposta

In [ ]:
from mimesis import Generic, Person, Address
from mimesis.builtins import BrazilSpecProvider
from random import random
from uuid import uuid4

# cria a classe pessoa física
class PF(Node):
	BrazilSpecProvider.Meta.name = 'brasil'
  # generic permite a geração de cpf e cpnj
	_generic = Generic('pt-br')
	_generic.add_provider(BrazilSpecProvider)
	_person = Person('pt-br')
	_address = Address('pt-br')

	def __init__(self):
		self.__type = 'PF'   
		self.id = uuid4()
		self.label ='Pessoa Física'
		self.nome = self._person.full_name()
		self.cpf = self._generic.brasil.cpf()
		self.idade = self._person.age()


In [ ]:
pf = PF()
print(pf.toRow(['id','label', 'nome','cpf', 'idade']))

83bbdabc-6339-4b14-a831-cb9b789be464|Pessoa Física|Xica Tonoli|631.516.515-00|66


#### Pessoa Jurídica

In [ ]:
from mimesis import Generic, Business, Address
from mimesis.builtins import BrazilSpecProvider
from random import random
from uuid import uuid4


class PJ(Node):
	BrazilSpecProvider.Meta.name = 'brasil'
	_generic = Generic('pt-br')
	_generic.add_provider(BrazilSpecProvider)
	_business = Business('pt-br')
	_address = Address('pt-br')

	def __init__(self):
		self.__type = 'PJ'
		self.label ='Pessoa Jurídica'
		self.id = uuid4()
		self.tipo = self._business.company_type()
		self.nome = self._business.company()
		self.cnpj = self._generic.brasil.cnpj()
	
		if random() < p_cnae :
			self.cnae_visado = True
		else:
			self.cnae_visado = False


In [ ]:
pj = PJ()
print(pj.nome + " - " + pj.cnpj + " - " + pj.label )

Alcoa - 45.121.275/3890-60 - Pessoa Jurídica


#### Unidade Bancária

In [ ]:
from mimesis import Generic, Business, Address
from random import random
from uuid import uuid4


class UB(Node):
	_address = Address('pt-br')

	def __init__(self):
		self.__type = 'UB'
		self.label ='Unidade Bancária'
		self.id = uuid4()
		self.latitude = self._address.latitude()
		self.longitude = self._address.longitude()

#### Transações

In [ ]:
from random import random
from numpy import random as npr
from math import ceil
from mimesis import Datetime, Numbers, Text, Business
from uuid import uuid4


class Transacao(Node):
	_datetime = Datetime()
	_numbers = Numbers()
	_text = Text()
	_business = Business()

	def __init__(self, sourceId, targetId):
		self.__type = 'Transacao'
		# todo: implementar tipos de transações

    # define id da transacao
		self.id = uuid4()
    # seta origem de destino
		self.origem = sourceId
		self.destino = targetId
    # gera data entre 2020 e 2015 (últimos 5 anos)
		self.data = self._datetime.date(start=2015, end=2020)
		self.hora = self._datetime.time()
    # gera valor randomico de acordo com probabilidade. ajustar aos valores do coaf
		if random() < p_valor:
			self.valor = self._numbers.float_number(10000, 1000000)
		else:
			self.valor = npr.exponential(100)


In [ ]:
trans = Transacao(pf.id, pj.id)
print("De: " + pf.nome + " - Para: " + pj.nome + " Data: " + trans.data.strftime("%m/%d/%Y") + " Hora: " + trans.hora.strftime("%H:%M:%S") + " Valor: " + str(trans.valor))

De: Xica Tonoli - Para: Alcoa Data: 11/02/2020 Hora: 04:24:10 Valor: 21.013413832318225


#### Padrões de Transações

In [ ]:
from random import randint, random
from datetime import timedelta

TransacoesHeader = ['id', 'origem', 'destino', 'data', 'hora', 'valor']

def int2str(val):
  if val < 10: return '0' + str(val)
  return str(val)

# gera delay em dias entre transações
def __updateCurrentDate(date, delays):
  return date + timedelta(days=delays['days'])
# gera delay em horas/min/seg
def __updateCurrentTime(time, delays):
  hval = int(time.hour) + delays['hours']
  mval = int(time.minute) + delays['minutes']
  sval = int(time.second) + delays['seconds']

  if hval < 24:
    time.replace(hour=hval)
  if mval < 60:
    time.replace(minute=mval)
  if sval < 60:
    time.replace(second=sval)

  return time

# gera delays de até 5 dias, incluindo horas, minutos, segundos
def __generateDelays():
  return {
    'days': randint(0, 4),
    'hours': randint(0, 12),
    'minutes': randint(0, 59),
    'seconds': randint(0, 59)
  }

# busca nós randomicos dentre indices participantes em relacionamentos
def __generateRandomIndex(nodes, indicesParticipantes):
  idx = randint(0, len(nodes) - 1)
  while idx in indicesParticipantes:
    idx = randint(0, len(nodes) - 1)
  indicesParticipantes.add(idx)
  return idx

def generateFlowPattern(nodes):
  delays = __generateDelays()
  Transacoes = []
  indicesParticipantes = set()
  patternStructure = []

  currentDate = None
  currentTime = None
  
  startIdx = __generateRandomIndex(nodes, indicesParticipantes)
  endIdx = __generateRandomIndex(nodes, indicesParticipantes)

  totalValor = 50000 * random() + 50000 * random() # Gera um valor total inicial a ser dissipado
  totalComissao = 0.1 * random() * totalValor # A comissão será no mínimo 10% do valor total

  numeroCamadas = randint(2, 6) # Número de camadas intermediárias entre o nó origem e o nó destino

  initialtotalComissaoPorCamada = totalComissao / numeroCamadas
  totalComissaoPorCamada = 0
  comissaoPaga = 0
  valorRestante = totalValor

  for layer in range(numeroCamadas):
    nodesPorCamada = randint(1, 8) # Número de nós por camada entre 1 e 8
    patternStructure.append([])

    if layer != numeroCamadas - 1:
      totalComissaoPorCamada -= 0.1 * random() * initialtotalComissaoPorCamada
      totalComissaoPorCamada += 0.1 * random() * totalComissaoPorCamada

      comissaoPaga += totalComissaoPorCamada
    else:
      totalComissaoPorCamada = totalComissao - comissaoPaga
    
    taxaPorNode = totalComissaoPorCamada / nodesPorCamada

    for layerNode in range(nodesPorCamada):
      rndMiddlewareIdx = __generateRandomIndex(nodes, indicesParticipantes)
      patternStructure[layer].append(nodes[rndMiddlewareIdx])

    if layer == 0:
      valor = valorRestante / len(patternStructure[0])
      i = 0

      for node in patternStructure[layer]:
        t = Transacao(nodes[startIdx], node)
        t.valor = valor

        if i == 0:
          currentDate = t.data
          currentTime = t.hora
        else:
          currentDate = __updateCurrentDate(currentDate, delays)
          currentTime = __updateCurrentTime(currentTime, delays)

        t.data = currentDate
        t.hora = currentTime

        Transacoes.append(t.toRow(TransacoesHeader))
        i += 1

    else:
      prevLayerLength = len(patternStructure[layer-1])
      currentLayerLength = len(patternStructure[layer])

      for nodeOrigem in patternStructure[layer-1]:
        for nodeDestino in patternStructure[layer]:
          valor = (valorRestante - totalComissaoPorCamada) / (prevLayerLength * currentLayerLength)
          t = Transacao(nodeOrigem, nodeDestino)
          t.valor = valor

          currentDate = __updateCurrentDate(currentDate, delays)
          currentTime = __updateCurrentTime(currentTime, delays)

          t.date = currentDate
          t.time = currentTime

          Transacoes.append(t.toRow(TransacoesHeader))

      valorRestante -= totalComissaoPorCamada
  
  lastLayerLength = len(patternStructure[numeroCamadas - 1])
  for node in patternStructure[numeroCamadas - 1]:
    valor = valorRestante / lastLayerLength
    t = Transacao(node, nodes[endIdx])
    t.valor = valor
    
    currentDate = __updateCurrentDate(currentDate, delays)
    currentTime = __updateCurrentTime(currentTime, delays)

    t.date = currentDate
    t.currentTime = currentTime

    Transacoes.append(t.toRow(TransacoesHeader))  

  return Transacoes
      
def generateCircularPattern(nodes):
  delays = __generateDelays()
  Transacoes = []
  indicesParticipantes = set()

  currentDate = None
  currentTime = None
  
  startIdx = randint(0, len(nodes) - 1)
  indicesParticipantes.add(startIdx)

  totalValor = 50000 * random() + 50000 * random() # gera um valor randomico total para ser difundido no círculo
  totalComissao = 0.1 * random() * totalValor # soma total das comissoes aos intermediarios
  valorRestante = totalValor

  order = randint(1, 8) # numero de nós que farão parte do círculo ligando origem e destino são randomizados entre 1 e 8 
  prevIdx = startIdx
  initialComissaoPerStep = totalComissao / order
  stepComissao = 0

  for step in range(order):
    rndMiddlewareIdx = __generateRandomIndex(nodes, indicesParticipantes)
    stepComissao += 0.1 * random() * initialComissaoPerStep
    stepComissao -= 0.1 * random() * stepComissao

    valor = valorRestante - stepComissao
    t = Transacao(nodes[prevIdx], nodes[rndMiddlewareIdx])
    t.valor = valor
    if step == 0:
      currentDate = t.data
      currentTime = t.hora
    else:
      currentDate = __updateCurrentDate(currentDate, delays)
      currentTime = __updateCurrentTime(currentTime, delays)

    t.date = currentDate
    t.time = currentTime

    Transacoes.append(t.toRow(TransacoesHeader))

    prevIdx = rndMiddlewareIdx
    valorRestante -= stepComissao

  stepComissao += 0.1 * random() * initialComissaoPerStep
  stepComissao -= 0.1 * random() * stepComissao

  valor = valorRestante - stepComissao
  t = Transacao(nodes[prevIdx], nodes[startIdx])
  t.valor = valor
  if step == 0:
    currentDate = t.data
    currentTime = t.hora
  else:
    currentDate = __updateCurrentDate(currentDate, delays)
    currentTime = __updateCurrentTime(currentTime, delays)

  t.date = currentDate
  t.time = currentTime

  Transacoes.append(t.toRow(TransacoesHeader))
  return Transacoes

def generateTimePattern(nodes):
  delays = __generateDelays()
  Transacoes = []
  indicesParticipantes = set()

  currentDate = None
  currentTime = None

  #obtem dois nós para representar origem e destino
  startIdx = __generateRandomIndex(nodes, indicesParticipantes)
  endIdx = __generateRandomIndex(nodes, indicesParticipantes)
  # randomiza o número de 5 a 50 transações 
  order = randint(5, 50)
  valor = 50000 * random()

  for i in range(order):
    t = Transacao(nodes[startIdx], nodes[endIdx])
    t.valor = valor

    if i == 0:
      currentDate = t.data
      currentTime = t.hora
    else:
      currentDate = __updateCurrentDate(currentDate, delays)
      currentTime = __updateCurrentTime(currentTime, delays)

    t.date = currentDate
    t.time = currentTime

    Transacoes.append(t.toRow(TransacoesHeader))


  return Transacoes


## Geração dos Dados

In [ ]:
import time

logFile = open(time.strftime("%H.%M.%S_%d-%m-%Y") + '.log', 'a+')

def writeBatch(file, batch): file.write('\n'.join(batch) + '\n')

def log(str = ''):
  line = time.strftime("%d/%m/%Y %H:%M:%S") + ": " + str
  print(line)
  logFile.write(line  + '\n')


#### Nós

In [ ]:
import os
import threading


PFCabecalho = ['id', 'nome', 'cpf', 'idade', 'label']

PJCabecalho = ['id', 'tipo', 'nome', 'cnpj','cnae_visado', 'label']

UBCabecalho = ['id', 'latitude', 'longitude', 'label']


def gerarModelo(count, file, header, Model, modelname, batchSize, verbose=True):
	try:
		os.remove(file)
	except OSError:
		pass

	with open(file, 'a') as file:
		batch = []

		file.write('|'.join(header) + '\n')
		for i in range(0, count):
			c = Model()
			batch.append(c.toRow(header))

			if verbose and i % batchSize == 0:
				log()
				log(str(i) + ' ' + modelname + ' de ' + str(count) + ' foram gerados')

			if len(batch) > batchSize:
				writeBatch(file, batch)
				batch = []

		writeBatch(file, batch)
		log()
		log('Total ' + modelname + ' de ' + str(count) + 'foram gerados')

def generateNodes(files, counts, batchSize):
	PFsProcess = threading.Thread(target=lambda : gerarModelo(
		counts["PF"],
		files["PF"],
		header=PFCabecalho,
		Model=PF,
		modelname='PF',
		batchSize=batchSize
	))
	PJsProcess = threading.Thread(target=lambda : gerarModelo(
		counts["PJ"],
		files["PJ"],
		header=PJCabecalho,
		Model=PJ,
		modelname='PJ',
		batchSize=batchSize
	))
	UBsProcess = threading.Thread(target=lambda : gerarModelo(
		counts["UB"],
		files["UB"],
		header=UBCabecalho,
		Model=UB,
		modelname='UB',
		batchSize=batchSize
	))

	PFsProcess.start()
	PJsProcess.start()
	UBsProcess.start()

	PFsProcess.join()
	PJsProcess.join()
	UBsProcess.join()


#### Arestas

In [ ]:
import csv
import os
import threading
import math
from random import random


def __contagemTransacoes():
	y = random()
	return math.ceil((-10/4) * math.log(y))
	
def __gerarArestas(nosOrigem, nosDestino, arquivoSaida, probabilidadeConexao, tamanhoBatch, label=''):
	try:
		os.remove(arquivoSaida)
	except OSError:
		pass

	numeroTotalArestas = 0
	with open(arquivoSaida, 'a') as file:
		batch = []
		contagemNosOrigem = 0

		for noOrigem in nosOrigem:
			contagemNosOrigem += 1
			arestasNoOrigem = [noOrigem, {}]

			if contagemNosOrigem % tamanhoBatch == 1:
				log(label + ': Numero Nó Origem ' + str(contagemNosOrigem) + ', número de arestas: ' + str(numeroTotalArestas))

			for noDestino in nosDestino:
				if noOrigem == noDestino: continue
				
				if random() < probabilidadeConexao:
					numeroConexoes = __contagemTransacoes()
					arestasNoOrigem[1][noDestino] = numeroConexoes
					numeroTotalArestas += 1

			arestasNoOrigem[1] = '"' + str(arestasNoOrigem[1]) + '"'
			batch.append('|'.join(arestasNoOrigem))

			if (len(batch) > tamanhoBatch):
				writeBatch(file, batch)
				batch = []

		writeBatch(file, batch)
		log(label + ' ### TOTAL de arestas ' + str(numeroTotalArestas))

def gerarArestas(files, probs, tamanhoBatch):
	print("Lendo nós ")
	pfs = set()
	pjs = set()
	ubs = set()

	with open(files['PF'], 'r') as f:
		reader = csv.reader(f, delimiter="|")
		next(reader)
		log("Carregando PFs...")
		for row in reader:
			pfs.add(row[0])

	with open(files['PJ'], 'r') as f:
		reader = csv.reader(f, delimiter="|")
		next(reader)
		log("Carregando PJs...")
		for row in reader:
			pjs.add(row[0])

	with open(files['UB'], 'r') as f:
		reader = csv.reader(f, delimiter="|")
		next(reader)
		log("Carregando unidades bancárias...")
		for row in reader:
			ubs.add(row[0])

	pfpfEdgesProcess = threading.Thread(target=lambda: __gerarArestas(
		pfs,
		pfs,
		files['pfs-pfs-arestas'],
		probs[0],
		tamanhoBatch,
		label='pf->pf'
	))
	pfpjEdgesProcess = threading.Thread(target=lambda: __gerarArestas(
		pfs,
		pjs,
		files['pfs-pjs-arestas'],
		probs[1],
		tamanhoBatch,
		label='pf->pj'
	))
	pfubEdgesProcess = threading.Thread(target=lambda: __gerarArestas(
		pfs,
		ubs,
		files['pfs-ubs-arestas'],
		probs[2],
		tamanhoBatch,
		label='pf->ub'
	))
	pjpfEdgesProcess = threading.Thread(target=lambda: __gerarArestas(
		pjs,
		pfs,
		files['pjs-pfs-arestas'],
		probs[3],
		tamanhoBatch,
		label='pj->pf'
	))

	pfpfEdgesProcess.start()
	pfpjEdgesProcess.start()
	pfubEdgesProcess.start()
	pjpfEdgesProcess.start()

	pfpfEdgesProcess.join()
	pfpjEdgesProcess.join()
	pfubEdgesProcess.join()
	pjpfEdgesProcess.join()


#### Transações

In [ ]:
import csv
import threading
import os


transacoesCabecalho = ['id', 'origem', 'destino', 'data', 'hora', 'valor']

def __generateTransacoes(arestas, transacoesFile, batchSize, label):
	try:
		os.remove(transacoesFile)
	except OSError:
		pass

	numeroTotalTransacoes = 0

	with open(transacoesFile, 'a') as transacoes:
		batch = []
		numeroNosOrigem = 0

		for origemNode, targets in arestas.items():
			numeroNosOrigem += 1
			if numeroNosOrigem % batchSize == 1: log(label + ": gerando transações do nó " + str(numeroNosOrigem) + ", número de transações: " + str(numeroTotalTransacoes))
			
			targetNodesCount = 0
			for targetNode, transacoesCount in targets.items():
				targetNodesCount += 1

				for i in range(0, transacoesCount):
					t = Transacao(origemNode, targetNode)

					batch.append(t.toRow(transacoesCabecalho))
					numeroTotalTransacoes += 1

				if len(batch) > batchSize:
					writeBatch(transacoes, batch)
					batch = []

		if len(batch) != 0:
			writeBatch(transacoes, 
		log(label + ": Totais: Gerando transações a partir do nó " + str(numeroNosOrigem) + ", Número de Transacoes: " + str(numeroTotalTransacoes))

def generateTransacoes(files, batchSize):
	print("Lendo arestas...")
	arestasPF = {}
	arestasPJ = {}

	with open(files['pfs-pfs-arestas'], 'r') as file:
		reader = csv.reader(file, delimiter="|")
		for row in reader:
			if not row[0] in arestasPF:
				arestasPF[row[0]] = {}
			arestasPF[row[0]].update(eval(row[1]))

	with open(files['pfs-pjs-arestas'], 'r') as file:
		reader = csv.reader(file, delimiter="|")
		for row in reader:
			if not row[0] in arestasPF:
				arestasPF[row[0]] = {}
			arestasPF[row[0]].update(eval(row[1]))

	with open(files['pfs-ubs-arestas'], 'r') as file:
		reader = csv.reader(file, delimiter="|")
		for row in reader:
			if not row[0] in arestasPF:
				arestasPF[row[0]] = {}
			arestasPF[row[0]].update(eval(row[1]))

	with open(files['pjs-pfs-arestas'], 'r') as file:
		reader = csv.reader(file, delimiter="|")
		for row in reader:
			if not row[0] in arestasPF:
				arestasPJ[row[0]] = {}
			arestasPJ[row[0]].update(eval(row[1]))

	origemPFTransacoes = threading.Thread(target = lambda: __generateTransacoes(
		arestasPF,
		files['pf-transacoes'],
		batchSize,
		label='transacao(pf->*)'
	))

	pjPFTransacoes = threading.Thread(target = lambda: __generateTransacoes(
		arestasPJ,
		files['pj-transacoes'],
		batchSize,
		label='transacao(pj->pf)'
	))

	origemPFTransacoes.start()
	pjPFTransacoes.start()

	origemPFTransacoes.join()
	pjPFTransacoes.join()



#### Padrões de Transações

In [ ]:
import csv
import threading
import os
from math import ceil
from random import random


cabecalhoTransacoes = ['id', 'origem', 'destino', 'data', 'hora', 'valor']


def __geraPadroes(nos, counts, arquivoTransacoes, batchSize, patternsGenerator, label):
	try:
		os.remove(arquivoTransacoes)
	except OSError:
		pass

	numeroTotalPadroes = 0
	numeroTotalTransacoes = 0


	numeroPadroes = ceil(0.1 * random() * counts['PF'])
	# numeroPadroes = 1

	log(label + ': Número de Padrões a serem gerados: ' + str(numeroPadroes))

	with open(arquivoTransacoes, 'a') as transacoes:
		batch = []

		# transacoes.write('|'.join(cabecalhoTransacoes) + '\n')
		for i in range(numeroPadroes):
			batch += patternsGenerator(nos)

			numeroTotalPadroes += 3
			numeroTotalTransacoes += len(batch)

			if len(batch) > batchSize:
				writeBatch(transacoes, batch)
				batch = []
				log(label + ': Gerados ' + str(numeroTotalPadroes) + ' padrões com  ' + str(numeroTotalTransacoes) + ' transacoes')

		if len(batch) != 0:
			writeBatch(transacoes, batch)

		log(label + ': Número Gerado ' + str(numeroTotalPadroes) + ' padrões com ' + str(numeroTotalTransacoes) + ' transacoes')


def geraPadroes(files, counts, batchSize):
	print("Lendo nós...")
	nos = []

	with open(files['PF'], 'r') as f:
		reader = csv.reader(f, delimiter="|")
		next(reader)
		log("Carregando PFs...")
		for row in reader:
			nos.append(row[0])

	with open(files['PJ'], 'r') as f:
		reader = csv.reader(f, delimiter="|")
		next(reader)
		log("Carregando PJs...")
		for row in reader:
			nos.append(row[0])

	# with open(files['UB'], 'r') as f:
	# 	reader = csv.reader(f)
	# 	next(reader)
	# 	log("Carregando UBs...")
	# 	for row in reader:
	# 		nos.add(row[0])
	fluxo = threading.Thread(target = lambda: __geraPadroes(
		nos,
		counts,
		files['padrao-fluxo-transacoes'],
		batchSize,
		generateFlowPattern,
		label='Padrão Fluxo'
	))

	circular = threading.Thread(target = lambda: __geraPadroes(
		nos,
		counts,
		files['padrao-circular-transacoes'],
		batchSize,
		generateCircularPattern,
		label='Padrão Circular'
	))

	iterativo = threading.Thread(target = lambda: __geraPadroes(
		nos,
		counts,
		files['padrao-iterativo-transacoes'],
		batchSize,
		generateTimePattern,
		label='Padrão Iterativo'
	))

	fluxo.start()
	circular.start()
	iterativo.start()

	fluxo.join()
	circular.join()
	iterativo.join()

# Gerar Arquivos

Serão gerados 3 arquivos com informações de nós (PF, PJ e UBs)
Serão gerados 4 arquivos com informações de relacionamentos, um com transações comuns e 3 referentes aos padrões de transações gerados.

In [ ]:
#criacao de pasta destino
#dataDir = time.strftime("%H.%M.%S_%d-%m-%Y")
#os.makedirs(dataDir)

#usando google drive
dataDir = "/content/drive/My Drive/Mestrado/estudo_dirigido_2/datasets"

#arquivos a serem gerados
files = {
	"PF" : dataDir + '/nos-pfs.csv',
	"PJ" : dataDir + '/nos-pjs.csv',
	"UB" : dataDir + '/nos-ubs.csv',

	"pfs-pfs-arestas": dataDir + '/pf-pf-arestas.csv',
	"pfs-pjs-arestas": dataDir + '/pf-pj-arestas.csv',
	"pfs-ubs-arestas": dataDir + '/pf-ub-arestas.csv',
	"pjs-pfs-arestas": dataDir + '/pj-pf-arestas.csv',

	"pf-transacoes": dataDir + '/pfs-originando-transacoes-nos.csv',
	"pj-transacoes": dataDir + '/pjs-originando-transacoes-nos.csv',

	"padrao-fluxo-transacoes": dataDir + '/padrao-fluxo-transacoes-nos.csv',
	"padrao-circular-transacoes": dataDir +  '/padrao-circular-transacoes-nos.csv',
	"padrao-iterativo-transacoes": dataDir + '/padrao-iterativo-transacoes-nos.csv'
}

log('Passos a executar: ' + str(steps))


# Geração de Nos
if 'nos' in steps:
	log()
	log('*****')
	log('Gerando nós....')
	generateNodes(files, counts, batchSize)

# gerando arestas
if 'arestas' in steps:
	log()
	log('*****')
	log('Gerando arestas...')
	gerarArestas(files, probs, batchSize)

# gerando transacoes
if 'transacoes' in steps:
	log()
	log('*****')
	log('Gerando transacoes')
	generateTransacoes(files, batchSize)

# gerando padroes
if 'patterns' in steps:
	log()
	log('*****')
	log('Gerando padroes')
	geraPadroes(files, counts, batchSize)


21/07/2020 15:21:13: Passos a executar: {'nos', 'arestas', 'patterns', 'transacoes'}
21/07/2020 15:21:13: 
21/07/2020 15:21:13: *****
21/07/2020 15:21:13: Gerando nós....
21/07/2020 15:21:14: 
21/07/2020 15:21:14: 0 PF de 10000 foram gerados
21/07/2020 15:21:14: 
21/07/2020 15:21:14: 0 UB de 50 foram gerados
21/07/2020 15:21:14: 
21/07/2020 15:21:14: Total UB de 50foram gerados
21/07/2020 15:21:14: 
21/07/2020 15:21:14: 0 PJ de 500 foram gerados
21/07/2020 15:21:14: 
21/07/2020 15:21:14: Total PJ de 500foram gerados
21/07/2020 15:21:14: 
21/07/2020 15:21:14: Total PF de 10000foram gerados
21/07/2020 15:21:14: 
21/07/2020 15:21:14: *****
21/07/2020 15:21:14: Gerando arestas...
Lendo nós 
21/07/2020 15:21:14: Carregando PFs...
21/07/2020 15:21:14: Carregando PJs...
21/07/2020 15:21:14: Carregando unidades bancárias...
21/07/2020 15:21:14: pf->pf: Numero Nó Origem 1, número de arestas: 0
21/07/2020 15:21:14: pf->ub: Numero Nó Origem 1, número de arestas: 0
21/07/2020 15:21:14: pf->pj: Num

In [ ]:
import os
import glob
import pandas as pd
os.chdir(dataDir)
extension = 'csv'
all_filenames = [i for i in glob.glob('*transacoes*.{}'.format(extension))]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_csv.to_csv( "transacoes.csv", index=False, encoding='utf-8-sig')

#os arquivos de transacoes são


## Comandos para Neo4j

#### Cabeçalhos

##### Transações

`id:string|origem:START_ID|destino:END_ID|data:date|hora:time|valor:float`

##### PFs

`id:ID|nome|cpf|idade|:LABEL`

##### PJs

`id:ID|tipo|nome|cnpj|cnae_visado|:LABEL`

##### UBs

`id:ID|latitude|longitude|:LABEL`

### Comando de importação

```
bin/neo4j-admin import \
      --nodes=import/nos-ubs.csv \
      --nodes=import/nos-pfs.csv \
      --nodes=import/nos-pjs.csv \
      --relationships=import/transacoes.csv \
      --delimiter "|"
```

### Exportando do Neo4J para GraphML

Basta efetuar a chamada em Cypher:

`CALL apoc.export.graphml.all("rede.graphml", {})`

### Resultado da Importação

```
Imported:
  102550 nodes
  3790033 relationships
  15572782 properties
Peak memory usage: 1.032GiB
```